# 神经网络初探----FizzBuzz 

FizzBuzz是一个游戏，规则：从 1开始，当遇到3的倍数输出fizz，当遇到5的倍数输出buzz，当遇到15的倍数输出fizzbuzz，quit他情况输出正常数值。

In [2]:
def fizz_buzz_encode(i):
    #对所有整数分成4类
    if i % 15 == 0:return 3
    elif i % 5 == 0:return 2
    elif i % 3 == 0:return 1
    else:return 0

In [4]:
def fizz_buzz_decode(i,prediction):
    #根据整数类别，输出其标签
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

In [16]:
import numpy as np

NUM_DIGITS = 10 # 维数或者是二进制数的长度

def binary_encode(i, num_digits):
    # 把整数i转化为10位的二进制数
    return np.array([i >> d & 1 for d in range(num_digits)][::-1])# 每次右移一位后与 1 做 与运算

In [15]:
binary_encode(8,NUM_DIGITS)

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

In [33]:
import torch
import torch.nn.functional as F     # 激励函数都在这
# 101 -- 1024 是训练数据，1--100是测试数据
trainX = torch.Tensor([binary_encode(i,NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
trainY = torch.LongTensor([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DIGITS)])

In [30]:
class Net(torch.nn.Module):     # 继承 torch 的 Module
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()     # 继承 __init__ 功能
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # 隐藏层线性输出 n_feature-->n_hidden
        self.out = torch.nn.Linear(n_hidden, n_output)       # 输出层线性输出 n_hidden -->n_output

    def forward(self, x):
        # 正向传播输入值, 神经网络分析出输出值
        x = self.hidden(x)
        x = F.relu(x)      # 激励函数(隐藏层的线性值)
        x = self.out(x)    # 输出值, 但是这个不是预测值, 预测值还需要再另外计算
        return x
fizzbuzzNet = Net(n_feature=NUM_DIGITS, n_hidden=100, n_output=4) # 10维的特征，100个隐藏节点，输出4个类别

In [82]:
# 训练代码
loss_fn = torch.nn.CrossEntropyLoss() # 损失函数
optimizer = torch.optim.SGD(fizzbuzzNet.parameters(),lr=3e-14)#优化器使用SGD,学习率为0.05

for t in range(10000):
    out = fizzbuzzNet(trainX)     # 喂给 net 训练数据 x, 输出分析值

    loss = loss_fn(out, trainY)     # 计算两者的误差
    if t % 200 == 0:
        print("Epoch",t,loss.item())
    optimizer.zero_grad()   # 清空上一步的残余更新参数值
    loss.backward()         # 误差反向传播, 计算参数更新值
    optimizer.step()        # 将参数更新值施加到 net 的 parameters 上

Epoch 0 0.17743371427059174
Epoch 200 0.17743371427059174
Epoch 400 0.17743371427059174
Epoch 600 0.17743371427059174
Epoch 800 0.17743371427059174
Epoch 1000 0.17743371427059174
Epoch 1200 0.17743371427059174
Epoch 1400 0.17743371427059174
Epoch 1600 0.17743371427059174
Epoch 1800 0.17743371427059174
Epoch 2000 0.17743371427059174
Epoch 2200 0.17743371427059174
Epoch 2400 0.17743371427059174
Epoch 2600 0.17743371427059174
Epoch 2800 0.17743371427059174
Epoch 3000 0.17743371427059174
Epoch 3200 0.17743371427059174
Epoch 3400 0.17743371427059174
Epoch 3600 0.17743371427059174
Epoch 3800 0.17743371427059174
Epoch 4000 0.17743371427059174
Epoch 4200 0.17743371427059174
Epoch 4400 0.17743371427059174
Epoch 4600 0.17743371427059174
Epoch 4800 0.17743371427059174
Epoch 5000 0.17743371427059174
Epoch 5200 0.17743371427059174
Epoch 5400 0.17743371427059174
Epoch 5600 0.17743371427059174
Epoch 5800 0.17743371427059174
Epoch 6000 0.17743371427059174
Epoch 6200 0.17743371427059174
Epoch 6400 0.17

In [85]:
# 测试 1--100
testX = torch.Tensor([binary_encode(i,NUM_DIGITS) for i in range(1,101)])
with torch.no_grad():
    testY = fizzbuzzNet(testX)
print(testY)

tensor([[ 2.5864e+00, -1.0911e+00,  1.3262e-01, -1.9602e+00],
        [ 2.4493e+00, -7.4004e-01, -3.3043e-01, -1.4112e+00],
        [ 1.5196e-01,  2.1546e+00, -1.4864e-01, -2.6522e+00],
        [ 3.9217e+00,  3.6387e-01, -4.4317e-01, -3.6312e+00],
        [ 3.7021e-01, -1.9632e+00,  3.8478e+00, -2.0750e+00],
        [-9.7813e-01,  2.6337e+00, -6.7159e-01, -1.1104e+00],
        [ 2.7196e+00, -1.8832e+00,  8.5822e-01, -1.7009e+00],
        [ 2.8924e+00,  9.5848e-01, -5.8907e-01, -3.4343e+00],
        [-6.0627e-02,  3.3987e+00, -7.9691e-01, -3.2911e+00],
        [-8.3711e-01, -3.0852e-01,  2.5363e+00, -1.4550e+00],
        [ 2.6390e+00, -4.6937e-01,  2.3063e-01, -2.7418e+00],
        [-3.3170e-01,  3.5847e+00, -1.3046e+00, -2.2652e+00],
        [ 3.6803e+00, -3.0089e-01, -2.2965e+00, -1.5228e+00],
        [ 3.2644e+00, -4.1584e-01, -2.0570e+00, -8.5372e-01],
        [-1.7527e+00, -6.7341e-01, -3.7954e-01,  2.2959e+00],
        [ 8.4721e-01,  3.8680e-01, -6.4889e-01, -8.4734e-01],
        

In [51]:
#testY.shape -->torch.Size([100, 4]) testY是一个100行4列的tensor
# testY.max(0) -->torch.return_types.max(values=tensor([4.4122, 4.2147, 3.4067, 2.1118]),indices=tensor([74, 24,  5, 15]))
#  求每一列中的最大值，所以返回的是 4个值

testY.max(1) #求每一行中的最大值，所以返回的是100个值

torch.return_types.max(
values=tensor([1.6846, 2.4487, 2.4016, 1.9154, 3.7136, 3.4067, 2.4206, 2.4705, 2.6585,
        3.3038, 2.3508, 2.4712, 3.3353, 3.3695, 3.0021, 2.1118, 0.8616, 2.0676,
        2.7803, 3.4052, 2.0854, 1.8954, 2.5587, 3.0864, 4.2147, 2.0892, 2.2708,
        2.2817, 2.7116, 3.6666, 1.3290, 1.6625, 2.0644, 2.3352, 2.3743, 1.7261,
        2.0532, 2.9386, 2.3915, 1.3365, 2.6123, 2.1192, 0.6295, 3.5307, 2.8820,
        1.7108, 1.2637, 2.2561, 3.2705, 2.7956, 1.1399, 2.1919, 3.3724, 3.3695,
        3.5088, 1.3085, 1.3772, 3.1695, 2.7550, 3.2853, 1.5525, 1.8214, 1.4934,
        2.6228, 3.2724, 2.2943, 2.1944, 3.0201, 3.9760, 2.2283, 2.5513, 3.0834,
        3.5167, 3.8993, 4.4122, 1.3217, 2.6203, 2.9573, 4.0808, 2.6718, 1.9092,
        1.0576, 2.9509, 2.7618, 3.4676, 3.1674, 3.4217, 0.5793, 2.8720, 3.7150,
        0.6856, 2.3891, 2.9461, 2.9026, 2.9118, 1.8519, 3.2329, 2.9714, 2.6641,
        1.4043]),
indices=tensor([3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1

In [66]:
testY.max(1)[1] # 每行最大值的索引，就是预测结果

tensor([3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0,
        1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0,
        1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0,
        1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2,
        1, 0, 0, 1])

In [67]:
predicts = zip(range(1,101),testY.max(1)[1].tolist())

In [68]:
print([fizz_buzz_decode(i, x) for i, x in predicts])

['fizzbuzz', '2', '3', 'fizz', '5', 'buzz', 'fizz', '8', '9', 'fizz', 'buzz', '12', 'fizz', '14', '15', 'fizzbuzz', '17', '18', 'fizz', '20', 'buzz', 'fizz', '23', '24', 'fizz', 'buzz', '27', 'fizz', '29', '30', 'fizzbuzz', '32', '33', 'fizz', '35', 'buzz', 'fizz', '38', '39', 'fizz', 'buzz', '42', 'fizz', '44', '45', 'fizzbuzz', '47', '48', 'fizz', '50', '51', 'fizz', '53', '54', 'fizz', '56', '57', 'fizz', '59', '60', 'fizzbuzz', '62', '63', 'fizz', '65', 'buzz', 'fizz', '68', '69', 'fizz', 'buzz', '72', 'fizz', '74', '75', 'fizzbuzz', '77', '78', 'fizz', '80', 'buzz', 'fizz', '83', '84', 'fizz', 'buzz', '87', 'fizz', '89', '90', 'fizzbuzz', '92', '93', 'fizz', '95', 'buzz', 'fizz', '98', '99', 'fizz']


In [86]:
predicts = testY.max(1)[1].tolist()
acc = 0
for i in range(1,101):
    if predicts[i-1] == fizz_buzz_encode(i):
        #print(i,"T")
        acc += 1
    else:
        print(i,"F-->",fizz_buzz_decode(i,predicts[i-1]),fizz_buzz_decode(i,fizz_buzz_encode(i)))
print("正确率为：%d%%"% acc)

50 F--> 50 buzz
55 F--> 55 buzz
正确率为：98%
